In [1]:
#Importa bibliotecas e remove acentos do dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
from sklearn.preprocessing import MinMaxScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers  import GRU
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

def accent_remover(x):
    try:
        float(x)
        return x
    except:
        return normalize('NFKD',x).encode('ASCII', 'ignore').decode('ASCII')

In [2]:
# Importa os datasets
dataset_2017 = pd.read_csv('datasets/datatran2017.csv', sep= ';', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset_2018 = pd.read_csv('datasets/datatran2018.csv', sep= ';', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset = pd.concat([dataset_2017,dataset_2018])

dataset = dataset.transform([accent_remover])
dataset.columns = dataset.columns.droplevel(1)

In [3]:
# Pega as dados das colunas:
# data_inversa	dia_semana	horario	uf	br	km	condicao_metereologica	latitude	longitude
training_set = dataset.iloc[:, [1,2,3,4,5,6,13,25,26]].values

In [4]:
# Converte os dados de string para number
encoder = LabelEncoder()
training_set[:,0] = encoder.fit_transform(training_set[:,0])
training_set[:,1] = encoder.fit_transform(training_set[:,1])
training_set[:,2] = encoder.fit_transform(training_set[:,2])
training_set[:,3] = encoder.fit_transform(training_set[:,3])
training_set[:,4] = encoder.fit_transform(training_set[:,4])
training_set[:,5] = encoder.fit_transform(training_set[:,5])
training_set[:,6] = encoder.fit_transform(training_set[:,6])
training_set[:,7] = encoder.fit_transform(training_set[:,7])
training_set[:,8] = encoder.fit_transform(training_set[:,8])

In [5]:
# Converte os dados para uma mesma faixa de valores de 0 a 1
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [6]:
# Cria a estrutura do dados com 60 timesteps e 1 saida
X_train = []
y_train = []
for i in range(60, 1258):
    X_train.append(training_set_scaled[i-60:i,:])
    y_train.append(training_set_scaled[i,:])
X_train, y_train = np.array(X_train), np.array(y_train)

In [7]:
# Inicializa a RNN
regressor = Sequential()

# Adiciona a primeira camada
regressor.add(GRU(units = 100, return_sequences = True, input_shape=(60,9)))
regressor.add(Dropout(0.2))

# Segunda camada
regressor.add(GRU(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Terceira camada
regressor.add(GRU(units = 100, return_sequences = True))
regressor.add(Dropout(0.2))

# Quarta camada
regressor.add(GRU(units = 100))
regressor.add(Dropout(0.2))

# Camada de saida
regressor.add(Dense(units = 9))

# Compila
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Executa o treinamento
regressor.fit(X_train, y_train, epochs = 50, batch_size = 32)

Epoch 1/50
38/38 [==============================] - 10s 253ms/step - loss: 0.0819
Epoch 2/50
38/38 [==============================] - 9s 245ms/step - loss: 0.0622
Epoch 3/50
38/38 [==============================] - 9s 244ms/step - loss: 0.0613
Epoch 4/50
38/38 [==============================] - 10s 252ms/step - loss: 0.0601
Epoch 5/50
38/38 [==============================] - 9s 245ms/step - loss: 0.0585
Epoch 6/50
38/38 [==============================] - 9s 245ms/step - loss: 0.0592
Epoch 7/50
38/38 [==============================] - 9s 246ms/step - loss: 0.0589
Epoch 8/50
38/38 [==============================] - 9s 245ms/step - loss: 0.0577
Epoch 9/50
38/38 [==============================] - 10s 257ms/step - loss: 0.0582
Epoch 10/50
38/38 [==============================] - 10s 255ms/step - loss: 0.0578
Epoch 11/50
38/38 [==============================] - 10s 251ms/step - loss: 0.0574
Epoch 12/50
38/38 [==============================] - 10s 254ms/step - loss: 0.0578
Epoch 13/50
38/38 [

In [15]:
# Testando os dados analisados com os de 2019
dataset_test = pd.read_csv('datasets/datatran2019.csv', sep= ';', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset_test = dataset_test.transform([accent_remover])
dataset_test.columns = dataset_test.columns.droplevel(1)

real_data = dataset_test.iloc[:, [1,2,3,4,5,6,13,25,26]].values
encoder = LabelEncoder()
real_data[:,0] = encoder.fit_transform(real_data[:,0])
real_data[:,1] = encoder.fit_transform(real_data[:,1])
real_data[:,2] = encoder.fit_transform(real_data[:,2])
real_data[:,3] = encoder.fit_transform(real_data[:,3])
real_data[:,4] = encoder.fit_transform(real_data[:,4])
real_data[:,5] = encoder.fit_transform(real_data[:,5])
real_data[:,6] = encoder.fit_transform(real_data[:,6])
real_data[:,7] = encoder.fit_transform(real_data[:,7])
real_data[:,8] = encoder.fit_transform(real_data[:,8])

In [16]:
# Realiza a predicao dos dados
inputs = real_data
inputs = sc.fit_transform(inputs)
X_test = []
for i in range(60, 80):
    X_test.append(inputs[i-60:i,:])
X_test = np.array(X_test)
predicted_data = regressor.predict(X_test)
predicted_data = sc.inverse_transform(predicted_data)

In [27]:
## ANN
# Selecionando dataset Treinamento
X = []
y = []
for i in predicted_data:
	X.append(i)
	y.append(1)

X_train = np.array(X)
y_train = np.array(y)

In [28]:
# Converte os dados para uma mesma faixa de valores de 0 a 1
sc = MinMaxScaler(feature_range = (0, 1))
X_train = sc.fit_transform(X_train)

In [32]:
# Cria a ANN
classifier = Sequential()
classifier.add(Dense(units = 9, kernel_initializer = 'uniform', activation = 'relu', input_dim=9))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 1.0000
Epoch 2/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6925 - accuracy: 1.0000
Epoch 3/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6920 - accuracy: 1.0000
Epoch 4/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6914 - accuracy: 1.0000
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 0.6908 - accuracy: 1.0000
Epoch 6/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6902 - accuracy: 1.0000
Epoch 7/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6897 - accuracy: 1.0000
Epoch 8/100
1/1 [==============================] - 0s 4ms/step - loss: 0.6891 - accuracy: 1.0000
Epoch 9/100
1/1 [==============================] - 0s 2ms/step - loss: 0.6885 - accuracy: 1.0000
Epoch 10/100
1/1 [==============================] - 0s 6ms/step - loss: 0.6879 - accuracy: 1.0000
Epoch 11/100
1/1 [===========

1/1 [==============================] - 0s 2ms/step - loss: 0.5929 - accuracy: 1.0000
Epoch 85/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5904 - accuracy: 1.0000
Epoch 86/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5878 - accuracy: 1.0000
Epoch 87/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5852 - accuracy: 1.0000
Epoch 88/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5825 - accuracy: 1.0000
Epoch 89/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5798 - accuracy: 1.0000
Epoch 90/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5771 - accuracy: 1.0000
Epoch 91/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5743 - accuracy: 1.0000
Epoch 92/100
1/1 [==============================] - 0s 1ms/step - loss: 0.5714 - accuracy: 1.0000
Epoch 93/100
1/1 [==============================] - 0s 2ms/step - loss: 0.5685 - accuracy: 1.0000
Epoch 94/100
1/1 [===============

In [30]:
# Entrada dos dados do usuario
test = []
test.append(input("Digite a data_inversa:\nEx:'2019-01-01'\n"))
test.append(input("Digite o dia_semana:\nEx:'terça-feira'\n"))
test.append(input("Digite a horario:\nEx:'01:30:00'\n"))
test.append(input("Digite a uf:\nEx:'SP'\n"))
test.append(input("Digite a br:\nEx:'116'\n"))
test.append(input("Digite a km:\nEx:'218'\n"))
test.append(input("Digite a condicao_metereologica:\nEx:'Céu Claro'\n"))
test.append(input("Digite a latitude:\nEx:'-23,46052014'\n"))
test.append(input("Digite a longitude:\nEx:'-46,48772478'\n"))

test = pd.DataFrame({'dados': test})
test = encoder.fit_transform(test.dados)
test = np.reshape(test, (-1, 1))
test = sc.fit_transform(test[:])
test = np.reshape(test, (1, -1))

Digite a data_inversa:
Ex:'2019-01-01'
2020-11-20
Digite o dia_semana:
Ex:'terça-feira'
sexta-feira
Digite a horario:
Ex:'01:30:00'
12:00:00
Digite a uf:
Ex:'SP'
DF
Digite a br:
Ex:'116'
101
Digite a km:
Ex:'218'
343
Digite a condicao_metereologica:
Ex:'Céu Claro'
Céu Claro
Digite a latitude:
Ex:'-23,46052014'
-28.5197545
Digite a longitude:
Ex:'-46,48772478'
-49.0560047


In [31]:
# Predicao
y_pred = classifier.predict(test)
value = round(float(y_pred[0][0])*100, 2)
print("\n\n---> Probabilidade de ocorrer acidente: ", value)



---> Probabilidade de ocorrer acidente:  57.02
